# TP Auto-encodeur variationnel
Dans ce TP, vous allez implémenter un auto-encodeur variationnel (VAE). Afin de pouvoir visualiser les différentes densités de probabilité présentes dans la théorie d'un VAE, nous allons considérer un problème en 1D. Le fait de considérer un exemple 1D nous permettra également d'effectuer des apprentissages sur CPU en quelques secondes. L'implémentation se fera en PyTorch.

## Configuration
### Si vous utilisez un ordinateur de l'Enseirb:
#### 1) Lancer une session linux (et non pas windows)
#### 2) Aller dans "Applications", puis "Autre", puis "conda_pytorch" (un terminal devrait s'ouvrir)
#### 3) Dans ce terminal, taper la commande suivante pour lancer Spyder :  
`spyder &`  
#### 4) Configurer Spyder en suivant ces instructions [Lien configuration Spyder](https://gbourmaud.github.io/files/configuration_spyder_annotated.pdf).
### Si vous utilisez votre ordinateur personnel, il faudra installer Spyder.

### Créez un nouveau script python et copiez/collez le code suivant :

In [1]:
import math
import torch as t
import matplotlib.pyplot as plt
import torch.nn as nn

### Génération des données en 1D
L'objectif du VAE que nous implémentons est d'apprendre à générer de nouveaux échantillons qui ressemblent aux échantillons de la base de données $\{x_i\}_{i=1...N}$. Dans un cas réel, ces échantillons sont supposés issus d'une densité de probabilité inconnue $p_{data}(x)$. Dans notre contexte, nous simulons ces données $\{x_i\}_{i=1...N}$ ainsi nous avons besoin de définir $p_{data}(x)$. Nous considérerons un mélange de gaussiennes : $p_{data}(x)=\sum_{j=1...C} w_j \mathcal{N}(x;\mu_j,\sigma_j)$. Définissons les paramètres de ce mélange :

In [2]:
w_list = t.tensor([0.2, 0.45, 0.35, 0.1])
w_list /=w_list.sum() 

mu_list = t.tensor([-3., 2.5, 1.5, -2.5])
sigma_list = t.tensor([0.3, 1.2, 0.3, 0.2])

Dans le but d'afficher $p_{data}(x)$, définissons la fonction `plot_pdata` et effectuons un affichage :

In [ ]:
def plot_pdata(w_list, mu_list, sigma_list, ax=0, orientation='vertical'):
    
    x = t.linspace(start=-6., end=8., steps=1000)
    
    p_data = t.zeros_like(x)
    for i in range(w_list.shape[0]):
        p_data_i = (1./(sigma_list[i]*math.sqrt(2*math.pi))*t.exp(-0.5*((x-mu_list[i])/sigma_list[i])**2))
        p_data += w_list[i]*p_data_i
        
    if(ax==0):
        if(orientation=='vertical'):
            plt.plot(x,p_data,'k')
        else:
            plt.plot(p_data,x,'k')
    else:
        if(orientation=='vertical'):
            ax.plot(x,p_data,'k')
        else:
            ax.plot(p_data,x,'k')
    return

plt.figure(1)
plot_pdata(w_list, mu_list, sigma_list)

Nous pouvons désormais échantillonner notre base de données $\{x_i\}_{i=1...N}$ et afficher son histogramme :

In [ ]:
def sample_from_pdata(N, w_list, mu_list, sigma_list):
    
    n_c = w_list.shape[0]

    samp = t.zeros((N,1))
    mask = t.multinomial(w_list,num_samples=N,replacement=True)
    
    for i in range(n_c):
        samp_i = t.normal(mean=mu_list[i], std=sigma_list[i], size=(N,1))
        samp[mask==i] = samp_i[mask==i]
 
    return samp

N_samp = int(2e5)
X = sample_from_pdata(N_samp, w_list, mu_list, sigma_list)
nbins = 100
n, bins, patches = plt.hist(X.numpy(), nbins, density=True, facecolor='r', alpha=0.75)
plt.pause(0.1)

### Implémentation de l'encodeur
Afin de pouvoir afficher les densités de probabilité, nous considérerons un espace latent de dimension 1. Ainsi l'encodeur est un réseau de neurones dont l'entrée est un scalaire $x$ et dont les sorties sont deux scalaires, les paramètres de $q_m (z|x,\phi)=\mathcal{N}(x;\mu_{z|x},\sigma_{z|x}^2)$. Pour des raisons de stabilité numérique qui deviendront claires par la suite, au lieu de prédire l'écart-type $\sigma_{z|x}$, le réseau prédira le logarithme de la variance $\alpha_{z|x}=\ln (\sigma_{z|x}^2)$.

In [ ]:
class encodeur(nn.Module):
    def __init__(self,H,zDim):
        super(encodeur, self).__init__()
        self.H = H
        self.zDim = zDim
        self.linearIn = nn.Linear(1, H)
        self.activIn = nn.Tanh()
        
        self.linearHidden = nn.Linear(H, H)
        self.activHidden = nn.Tanh()
        
        self.linearOut = nn.Linear(H, 2*zDim)
 
    def forward(self, x):

        out = self.linearIn(x)
        out = self.activIn(out)
        
        out = self.linearHidden(out)
        out = self.activHidden(out)
        
        out = self.linearOut(out)

        mu_z = out[:,:self.zDim]
        logvar_z = out[:,self.zDim:]
        
        return mu_z, logvar_z

### Travail : Dessiner le schéma de l'encodeur sur une feuille

### Implémentation du décodeur
Le décodeur est un réseau de neurones dont l'entrée est un scalaire $z$ et dont la sortie est également un scalaire, à savoir la moyenne de $p_m (x|z,\theta)=\mathcal{N}(z;\mu_{x|z},\sigma_{x|z}^2)$. Dans notre cas, l'écart-type $\sigma_{x|z}$ n'est pas une sortie du décodeur mais un hyperparamètre qu'il va falloir régler manuellement. Nous reviendrons sur ce point par la suite.

In [ ]:
class decodeur(nn.Module):
    def __init__(self,H,zDim):
        super(decodeur, self).__init__()
        
        self.H = H
        self.zDim = zDim
        
        self.linearIn = nn.Linear(zDim, H)
        self.activIn = nn.Tanh()
        
        self.linearHidden = nn.Linear(H, H)
        self.activHidden = nn.Tanh()
        
        self.linearOut = nn.Linear(H, 1)
 
    def forward(self, x):

        out = self.linearIn(x)
        out = self.activIn(out)
        
        out = self.linearHidden(out)
        out = self.activHidden(out)
        
        mu_x = self.linearOut(out)

        return mu_x

### Travail : Dessiner le schéma du décodeur sur une feuille

### Définition des hyperparamètres

In [ ]:
H = 100
zDim = 1
learning_rate = 1e-3
batchSize = 256
inverseVarianceLikelihood = 5e2

### Instantiation de l'encodeur, du décodeur et de l'optimiseur

In [ ]:
enc = encodeur(H,zDim)
dec = decodeur(H,zDim)
optimizer = t.optim.Adam(list(enc.parameters()) + list(dec.parameters()), lr=learning_rate)   

### Apprentissage
Nous pouvons créer une boucle qui va implémenter les itérations de la descente de gradient stochastique (ainsi qu'une figure que nous utiliserons pour afficher l'évolution de l'apprentissage).

In [ ]:
NItMax = 40000
fig = plt.figure(figsize=(12, 4))
for i in range(NItMax):

Tirons un minibatch et encodons-le

In [ ]:
perm = t.randperm(N_samp)
X_batch = X[perm[:batchSize],:]
mu_z, logvar_z = enc.forward(X_batch)

Il nous faut maintenant tirer des échantillons de $p_m (x|z,\theta)=\mathcal{N}(z;\mu_{x|z},\sigma_{x|z}^2)$

In [ ]:
eps_samp = t.normal(mean=0.,std=1.,size=logvar_z.shape)
z_samp = mu_z + (logvar_z*0.5).exp_()*eps_samp

Décodons ces échantillons

In [ ]:
mu_x = dec.forward(z_samp)

L'objectif du VAE est de minimiser la divergence de Kullback-Leibler $KL(p_{data}(x)q_{m}(z|x, \phi)||p(z)p_{m}(x|z, \theta ))$ par rapport à $\phi$ et $\theta$. Après simplification, cette fonction de coût devient : $\mathcal{L}(\theta,\phi) = KL(\mathcal{N}(z;\mu_{z|x},\sigma_{z|x}^2)||\mathcal{N}(z;0,1)) -\ln(\mathcal{N}(x;\mu_{x|z},\sigma_{x|z}^2))+\text{cst}_{\phi,\theta}$.
Le premier terme est la divergence de Kullback-Leibler entre deux gaussiennes qui a pour expression $\frac{1}{2} (\sigma_{z|x}^2 + \mu_{z|x}^2 -1 -\ln(\sigma_{z|x}^2))$. Nous voyons que le dernier terme est le logarithme de la variance ce qui peut poser des problèmes de stabilité numérique. Ce terme explique la paramétrisation choisie pour la sortie de l'encodeur $\alpha_{z|x}=\ln (\sigma_{z|x}^2)$.

In [ ]:
KLD = 0.5*(logvar_z.exp() - logvar_z - 1 + (mu_z**2)).sum()
KLD /= logvar_z.shape[0]

Le second terme est l'opposé du logarithme d'une gaussienne : $-\ln(\mathcal{N}(x;\mu_{x|z},\sigma_{x|z}^2)=\frac{1}{2\sigma_{x|z}^2}(x-\mu_{x|z})^2 + \text{cst}$. Ici le coefficient $\frac{1}{\sigma_{x|z}^2}$ correspond à l'hyperparamètre `inverseVarianceLikelihood` défini plus haut.

In [ ]:
likelihood = (0.5*inverseVarianceLikelihood*(X_batch-mu_x)**2).sum()
likelihood /= logvar_z.shape[0]

In [ ]:
l = KLD+likelihood

Terminons une itération de descente de gradient stochastique en calculant les gradients et en mettant à jour les paramètres

In [ ]:
optimizer.zero_grad()
l.backward()
optimizer.step()

### Question 
D'après l'équation de la fonction de coût, quel comportement aura le VAE si l'hyperparamètre `inverseVarianceLikelihood` (coefficient $\frac{1}{\sigma_{x|z}^2}$) vaut $0$ ? Et si `inverseVarianceLikelihood` (coefficient $\frac{1}{\sigma_{x|z}^2}$) a une valeur très grande ? 

### Affichages
Afin de contrôler l'évolution de l'apprentissage nous pouvons afficher des informations dans le terminal

In [ ]:
print('It {} : loss : {:.2e}, KLD : {:.2e}, likelihood : {:.2e}, lr : {}'.format(i, l.item(), KLD.item(), likelihood.item(), optimizer.param_groups[0]['lr']))

et effectuer des affichages régulièrement dans une figure

In [ ]:
if((i<1000 and i%200==0) or i%2000 == 0):
        with t.no_grad():
            plt.clf()
            mu_z_v, logvar_z_v = enc.forward(X)
            eps_samp_v = t.normal(mean=0.,std=1.,size=logvar_z_v.shape)
            z_samp_v = mu_z_v + (logvar_z_v*0.5).exp_()*eps_samp_v
            x_samp_v = dec.forward(z_samp_v)
            
            new_Z_v = t.normal(mean=0.,std=1.,size=[10000,zDim])
            new_mu_v = dec.forward(new_Z_v)
            eps_samp_v = t.normal(mean=0.,std=1.,size=new_mu_v.shape)
            new_X_v = new_mu_v + (1./(math.sqrt(inverseVarianceLikelihood)))*eps_samp_v
            
            plotHistograms(fig, w_list, mu_list, sigma_list, nbins, X, mu_z_v, z_samp_v, x_samp_v, new_Z_v, new_mu_v, new_X_v)
            plt.pause(0.7)

Le code précédent fait appel à la fonction `plotHistograms` que voici (à mettre en haut du script) :

In [ ]:
def plotHistograms(fig, w_list, mu_list, sigma_list, nbins, X, mu_z, z_samp, x_samp, new_Z, new_mu_X, new_X):
    left, width = 0.1, 0.65/3
    bottom, height = 0.1, 0.65
    
    hist_height = 0.2/3
    
    spacing = 0.02
    
    rect_plt_train_enc = [left+hist_height, bottom+hist_height, width, height]
    rect_histx_train_enc = [left+hist_height, bottom , width, hist_height]
    rect_histy_train_enc = [left , bottom+hist_height, hist_height, height]
    
    rect_plt_train_dec = [width +hist_height+ spacing+left+hist_height, bottom+hist_height, width, height]
    rect_histx_train_dec = [width + hist_height+spacing+left+hist_height, bottom , width, hist_height]
    rect_histy_train_dec = [width + hist_height+spacing+left , bottom+hist_height, hist_height, height]
    
    rect_plt_train_gen = [2*(width +hist_height+ spacing)+left+hist_height, bottom+hist_height, width, height]
    rect_histx_train_gen = [2*(width + hist_height+spacing)+left+hist_height, bottom , width, hist_height]
    rect_histy_train_gen = [2*(width + hist_height+spacing)+left , bottom+hist_height, hist_height, height]
        
    #afficache encodeur données apprentissage
    
    ax = fig.add_axes(rect_plt_train_enc)
    ax_histx = fig.add_axes(rect_histx_train_enc, sharex=ax)
    ax_histx.tick_params(left = False, right = False , labelleft = False ,
                    labelbottom = False, bottom = False)
    ax_histx.invert_yaxis()
    
    ax_histy = fig.add_axes(rect_histy_train_enc, sharey=ax)
    ax_histy.tick_params(left = False, right = False , labelleft = False ,
                    labelbottom = False, bottom = False)
    ax_histy.invert_xaxis()
    
    plot_pdata(w_list, mu_list, sigma_list,ax=ax_histx);
    ax_histx.hist(X.numpy(), nbins, density=True, facecolor='r', alpha=0.75)
    
    plot_pdata(t.tensor([1.]), t.tensor([0.]), t.tensor([1.]), ax=ax_histy, orientation='horizontal');
    ax_histy.hist(z_samp.numpy(), nbins, density=True, facecolor='g', alpha=0.75, orientation='horizontal')
    
    ax.scatter(X[::10],z_samp[::10], s=0.1 , label=r'échantillons de $p_{data}(x)q_{m}(z|x, \phi )$')
    
    #ax.plot(X[::100], mu_z[::100], label='y=decodeur(x)')
    ax.grid('on')
    ax.legend()
    
    ax.set_title('Encodage données entraînement')
    
    #afficache décodeur données apprentissage
    ax = fig.add_axes(rect_plt_train_dec)
    ax_histx = fig.add_axes(rect_histx_train_dec, sharex=ax)
    ax_histx.tick_params(left = False, right = False , labelleft = False ,
                    labelbottom = False, bottom = False)
    ax_histx.invert_yaxis()
    
    ax_histy = fig.add_axes(rect_histy_train_dec, sharey=ax)
    ax_histy.tick_params(left = False, right = False , labelleft = False ,
                    labelbottom = False, bottom = False)
    ax_histy.invert_xaxis()
    
    plot_pdata(w_list, mu_list, sigma_list,ax=ax_histx, );
    ax_histx.hist(x_samp.numpy(), nbins, density=True, facecolor='r', alpha=0.75)
    
    plot_pdata(t.tensor([1.]), t.tensor([0.]), t.tensor([1.]), ax=ax_histy, orientation='horizontal');
    ax_histy.hist(z_samp.numpy(), nbins, density=True, facecolor='g', alpha=0.75, orientation='horizontal')
    
    x_samp_s,ind = t.sort(x_samp[::100],dim=0)
    z_samp_temp = z_samp[::100]
    z_samp_s = z_samp_temp[ind.view(-1)]
    ax.plot(x_samp_s, z_samp_s, label='encodeur(y)=x')
    ax.grid('on')
    ax.legend()

    ax.set_title('Décodage données entraînement')
             
    #afficache décodeur génération nouvelles données
    ax = fig.add_axes(rect_plt_train_gen)
    ax_histx = fig.add_axes(rect_histx_train_gen, sharex=ax)
    ax_histx.tick_params(left = False, right = False , labelleft = False ,
                    labelbottom = False, bottom = False)
    ax_histx.invert_yaxis()
    
    ax_histy = fig.add_axes(rect_histy_train_gen, sharey=ax)
    ax_histy.tick_params(left = False, right = False , labelleft = False ,
                    labelbottom = False, bottom = False)
    ax_histy.invert_xaxis()

    plot_pdata(w_list, mu_list, sigma_list,ax=ax_histx);
    ax_histx.hist(new_X.numpy(), nbins, density=True, facecolor='b', alpha=0.75)
    
    plot_pdata(t.tensor([1.]), t.tensor([0.]), t.tensor([1.]), ax=ax_histy, orientation='horizontal');
    ax_histy.hist(new_Z.numpy(), nbins, density=True, facecolor='c', alpha=0.75, orientation='horizontal')
    
    ax.scatter(new_X[::10],new_Z[::10], s=0.1 , label=r'échantillons de $p(z)p_{m}(x|z, \theta )$')
    #ax.plot(new_X[::10],new_Z[::10], label='encodeur(y)=x')
    ax.grid('on')
    ax.legend()

    ax.set_title('Génération de nouvelles données')

Vous pouvez lancer un apprentissage et observer le comportement du VAE au cours des itérations. Sur la figure, pouvez-vous identifier l'impact de la valeur de l'hyperparamètre `inverseVarianceLikelihood` (coefficient $\frac{1}{\sigma_{x|z}^2}$) ?

Vous pouvez faire varier les valeurs des hyperparamètres, modifier les architectures de l'encodeur et du décodeur, ou encore changer la forme de $p_{data}$.

S'il vous reste du temps, vous pouvez tenter d'implémenter un GAN pour résoudre le même problème.